In [1]:
import pandas as pd

# import sys
# sys.path.append("../../src/wikipedia")

# from wikipedia_parser import process_wikipedia_dump


# dump_file = "../../data/wikipedia/is/iswiki-20240901-pages-articles.xml"
# df = process_wikipedia_dump(dump_file, max_pages=None)
# df.to_parquet("iswiki.processed_datadump.parquet")

df = pd.read_parquet("iswiki.processed_datadump.parquet")

print(len(df))
df.head()


86163


,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text
0,3,Finnland,https://is.wikipedia.org/wiki/Finnland,4306,553,0,,110,47,2024-04-18T10:18:03Z,"\nFinnland (finnska Suomi; sænska: Finland), f...",{{Land\n| nafn = Lýðveldið Finnland\n| nafn_á_...
1,4,Forsíða,https://is.wikipedia.org/wiki/Fors%C3%AD%C3%B0a,28,5,0,,11,0,2021-01-02T00:52:18Z,\n\n\n\n\n\n\n\nGrein mánaðarins\n\nEldri grei...,"<templatestyles src=""Template:Forsíða/styles.c..."
2,6,Reykjavík,https://is.wikipedia.org/wiki/Reykjav%C3%ADk,2323,309,0,,51,41,2024-04-09T12:51:22Z,"\n\nReykjavík er höfuðborg Íslands, fjölmennas...",{{Um|höfuðborg Íslands|aðrar merkingar orðsins...
3,7,Svíþjóð,https://is.wikipedia.org/wiki/Sv%C3%AD%C3%BEj%...,3025,360,0,,44,56,2024-08-12T16:35:20Z,"\n\nSvíþjóð (sænska: Sverige), formlegt heiti ...",{{Land \n|nafn = Konungsríkið Svíþjóð\n|nafn_á...
4,591,Danmörk,https://is.wikipedia.org/wiki/Danm%C3%B6rk,4711,489,0,,55,58,2024-07-18T11:56:58Z,\nDanmörk (danska: Danmark; ) er land í Norður...,{{Land\n| nafn = Konungsríkið Danmörk\n| nafn_...


In [2]:
# df_pageviews = pd.read_parquet("202309_monthly_views.parsed.parquet")

pageviews_file = "/Users/einar/git/hafsteinn/together_rag/data/wikipedia_meta/aggregated/is.wikipedia_monthly_views.parquet"
df_pageviews = pd.read_parquet(pageviews_file)
# print(len(df_pageviews))

print(f"Number of pageviews rows: {len(df_pageviews):,}")

df_pageviews.head(2)


Number of pageviews rows: 1,182,149


,wiki_code,article_title,views,page_ids
0,is.wikipedia,"""\""Weird_Al\""_Yankovic""",1,[52652]
1,is.wikipedia,$,2,[30714]


**have almos 1.2 million rows in the pageviews data.**

~~This is interesting since according to the frontpage of is.wikipedia.com we have almost **60K** articles:~~  
This is because we first need to group by the article title, have many entries of the same title.

```
Á hinni íslensku Wikipedíu eru nú 59.115 greinar.
```




## Lets try to transform to page_ids 

I.e. now we have article_title which can have a list of page_ids.

I want to see how it would look if we have page_id to a list of article titles instead..



In [5]:
df_pageviews.head(2)

,wiki_code,article_title,views,page_ids
0,is.wikipedia,"""\""Weird_Al\""_Yankovic""",1,[52652]
1,is.wikipedia,$,2,[30714]


In [6]:
from collections import defaultdict

d = defaultdict(list)

for index, row in df_pageviews.iterrows():
    for page_id in row.page_ids:
        # d[page_id].append(row.article_title)
        # also need the wiki_code and the views column
        d[page_id].append((row.wiki_code, row.article_title, row.views))


print(len(d))

105702


In [ ]:
# df_pageviews_by_page_id = pd.DataFrame(list(d.items()), columns=["page_id", "article_titles"]
df_pageviews_by_page_id = pd.DataFrame(d.values(), columns=["wiki_code", "article_title", "views"])
df_pageviews_by_page_id.head(2)


### Explore the data

In [7]:
df_pageviews = df_pageviews[df_pageviews.wiki_code == "is.wikipedia"]

print(len(df_pageviews))
df_pageviews.head(2)

1182149


,wiki_code,article_title,views,page_ids
0,is.wikipedia,"""\""Weird_Al\""_Yankovic""",1,[52652]
1,is.wikipedia,$,2,[30714]


### How many of the pages can we match by title

In [8]:
len(df_pageviews) - len(df)

1095986

There are more rows in the pageviews data, we should hopefully 
have data for each of the entries in the processed dump.



In [9]:
xpageviews_titles = set(df_pageviews.article_title)

print(f"Number of titles in data: {len(data_titles)}")
print(f"Number of titles in pageviews: {len(pageviews_titles)}")
print(f"Number of titles in both data: {len(data_titles.intersection(pageviews_titles))}")


Number of titles in data: 86163
Number of titles in pageviews: 202415
Number of titles in both data: 40861


In [10]:
df[~df.title.isin(df_pageviews.article_title)].head(2)

,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text
7,599,Föll í íslensku,https://is.wikipedia.org/wiki/F%C3%B6ll_%C3%AD...,148,11,0,,1,0,2023-09-12T13:13:18Z,"\n\nÍ íslensku eru fjögur föll, nefnifall, þol...",{{Föll í íslensku}}\n\nÍ [[Íslenska|íslensku]]...
9,602,Borgarstjóri Reykjavíkur,https://is.wikipedia.org/wiki/Borgarstj%C3%B3r...,683,191,0,,4,5,2024-08-16T13:27:12Z,\nBorgarstjóri Reykjavíkur er æðsti embættisma...,[[Mynd:20190624 CityHall 9152 (48463162702).jp...


So page_id 602 with title "Borgarstjóri Reykjavíkur" is not in the pageviews data.


In [7]:
df_pageviews[df_pageviews.article_title.str.contains("Borgarstjóri", case=False)]

,wiki_code,article_title,views,page_ids
28456220,is.wikipedia,Borgarstjóri,17,[51093]
28456221,is.wikipedia,Borgarstjóri_Reykjavíkur,78,"[602, null]"
28456222,is.wikipedia,Borgarstjórinn,4,[159481]
28517568,is.wikipedia,Páll_Einarsson_(borgarstjóri),16,[65350]


It is a simple case of pageviews data having _ instead of space. Also we might be able to use the page id

In [11]:
df_pageviews["title_match"] = df_pageviews.article_title.str.replace("_", " ").str.lower()
df["title_match"] = df.title.str.replace("_", " ").str.lower()

data_titles = set(df.title_match)
pageviews_titles = set(df_pageviews.title_match)

print(f"Number of titles in data: {len(data_titles)}")
print(f"Number of titles in pageviews: {len(pageviews_titles)}")
print(f"Number of titles in both data: {len(data_titles.intersection(pageviews_titles))}")
print(f"Number of unmatched titles in data: {len(data_titles) - len(data_titles.intersection(pageviews_titles))}")
print(f"Number of unmatched titles in pageviews: {len(pageviews_titles) - len(data_titles.intersection(pageviews_titles))}")


Number of titles in data: 85506
Number of titles in pageviews: 200264
Number of titles in both data: 85148
Number of unmatched titles in data: 358
Number of unmatched titles in pageviews: 115116


### still many unmatched titles

But way less than before, there are still many unmatched titles.

In [9]:
pd.set_option("display.max_colwidth", 100)
df[~df.title_match.isin(df_pageviews.title_match)].sort_values(by=["external_link_count"], ascending=False).head(5)

,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text,title_match
83325,177298,Forsetakjör á Íslandi 2024,https://is.wikipedia.org/wiki/Forsetakj%C3%B6r_%C3%A1_%C3%8Dslandi_2024,3485,306,0,,64,90,2024-07-06T11:59:01Z,"\nForsetakjör á Íslandi 2024 fór fram þann 1. júní 2024. Halla Tómasdóttir sigraði með 34,1% gil...",[[Mynd:Forsetakosningar 2024.svg|thumb|Niðurstöður kosninganna]]\n'''Forsetakjör á Íslandi 2024'...,forsetakjör á íslandi 2024
61256,141279,Borgarastyrjöldin í Jemen (2014–),https://is.wikipedia.org/wiki/Borgarastyrj%C3%B6ldin_%C3%AD_Jemen_%282014%E2%80%93%29,1827,172,0,,130,75,2024-08-30T23:20:10Z,\nBorgarastyrjöldin í Jemen eru átök sem hófust árið 2014 og eru tveir hópar sem berjast þar um ...,{{Infobox military conflict\n| conflict = Borgarastyrjöldin í Jemen\n| partof = [[Arabís...,borgarastyrjöldin í jemen (2014–)
85534,181102,Peter Pellegrini,https://is.wikipedia.org/wiki/Peter_Pellegrini,724,72,0,,40,61,2024-06-19T00:15:13Z,\nPeter Pellegrini (f. 6. október 1975) er slóvakískur stjórnmálamaður og núverandi forseti Slóv...,"{{Forseti\n| nafn = Peter Pellegrini\n| mynd = Peter Pellegrini, 2024 (cropped).jpg\n|...",peter pellegrini
84247,178822,Alsírstríðið,https://is.wikipedia.org/wiki/Als%C3%ADrstr%C3%AD%C3%B0i%C3%B0,2148,178,0,,126,59,2024-05-03T02:04:25Z,"\nAlsírstríðið, einnig kallað alsírska sjálfstæðisstríðið eða alsírska byltingin (arabíska: الثو...",{{Infobox military conflict\n| conflict = Alsírstríðið<br />{{Nobold|{{lang|ar|ثورة الت...,alsírstríðið
45794,106161,Forsetakjör á Íslandi 2012,https://is.wikipedia.org/wiki/Forsetakj%C3%B6r_%C3%A1_%C3%8Dslandi_2012,761,44,0,,19,52,2024-07-06T11:08:17Z,thumb|right|Híbýli forseta Íslands eru á Bessastöðum.\nForsetakjör á Íslandi 2012 fór fram lauga...,[[Mynd:OB090126c-3053_Bessastadir.JPG|thumb|right|Híbýli forseta Íslands eru á [[Bessastaðir|Bes...,forsetakjör á íslandi 2012


In [10]:
df.page_id.dtype

dtype('O')

In [11]:
unmatched_ids = df[~df.title_match.isin(df_pageviews.title_match)].page_id.unique()
len(unmatched_ids)

23919

In [12]:
type(df_pageviews.iloc[0].page_ids[0])

str

In [13]:
# df_pageviews[df_pageviews.page_id.isin(unmatched_ids)].head(2)
# the page_ids column in df_pageviews is a list of page ids, so we need to match them to the page_id in df

# def check_page_id_match(page_ids, unmatched_ids):
#     return any(str(id) in page_ids for id in unmatched_ids)

# matched_rows = df_pageviews[df_pageviews['page_ids'].apply(lambda x: check_page_id_match(x, unmatched_ids))]

# print(f"Number of matched rows: {len(matched_rows)}")
# matched_rows.head()

import numpy as np

# Convert unmatched_ids to a set of strings for faster lookup
unmatched_ids_set = set(map(str, unmatched_ids))

# Vectorized function to check for matches
def check_match(page_ids):
    return np.frompyfunc(lambda x: x in unmatched_ids_set, 1, 1)(page_ids).any()

# Apply the vectorized function
matched_mask = df_pageviews['page_ids'].apply(check_match)

# Get the matched rows
matched_rows = df_pageviews[matched_mask]

print(f"Number of matched rows: {len(matched_rows)}")
matched_rows.head()




Number of matched rows: 506


,wiki_code,article_title,views,page_ids,title_match
28446992,is.wikipedia,-,352,"[65519, 155031, 171906, 33641, 127265, 20264, 171929, 2453, 27685, 18332, 26113, 25992, 132275, ...",-
28449654,is.wikipedia,621_f.Kr.,1,[145150],621 f.kr.
28449656,is.wikipedia,622_f.Kr.,2,[145150],622 f.kr.
28449657,is.wikipedia,623_f.Kr.,1,[145150],623 f.kr.
28449659,is.wikipedia,626_f.Kr.,1,[145150],626 f.kr.


In [14]:
debug_ids = [65519, 155031, 171906, 33641, 127265, 20264, 171929, 2453, 27685, 18332, 26113, 25992, 132275
]
debug_ids = [str(id) for id in debug_ids]

df[df.page_id.isin(debug_ids)]







,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text,title_match
881,2453,Hannover,https://is.wikipedia.org/wiki/Hannover,2088,162,0,,12,0,2022-11-28T17:19:54Z,\nHannover er höfuðborg og stærsta borg þýska sambandslandsins Neðra-Saxlands með um 540.000 íbú...,{{Byggð\n| Nafn = Hannover\n| Mynd = Hannover Blick Neues Rathaus 01.jpg\n|alt=\n| Myndatexti =\...,hannover
6560,18332,Christina,https://is.wikipedia.org/wiki/Christina,785,2,0,,4,2,2023-08-07T00:11:27Z,\nChristina er íslenskt kvenmannsnafn.\n\n== Dreifing á Íslandi ==\n\n\nImageSize = width:600 he...,{{Íslenskt mannanafn\n| nafn = Christina\n| kyn = kvk\n| nefnifall = Christina\n| þolfall = Chri...,christina
8268,20264,Pokabjörn,https://is.wikipedia.org/wiki/Pokabj%C3%B6rn,330,38,0,,3,2,2022-03-26T04:50:17Z,\nPokabjörn (eða kóalabjörn) (fræðiheiti: Phascolarctos cinereus) er spendýr af pokadýraflokki s...,{{Taxobox\n| color=pink \n| name=Pokabjörn\n| status=LC\n| image = Koala climbing tree.jpg\n| im...,pokabjörn
10999,25992,Stonehenge,https://is.wikipedia.org/wiki/Stonehenge,168,21,0,,1,0,2024-01-31T10:46:26Z,"\n\nStonehenge er mannvirki frá nýsteinöld og bronsöld, gert úr risasteinum staðsett við Amesbur...",[[Mynd:Stonehenge-Green.jpg|thumb|250px|Stonehenge 2009]]\n[[Mynd:Stonehenge Total.jpg|thumb|Sto...,stonehenge
11065,26113,Flokkskvísl,https://is.wikipedia.org/wiki/Flokkskv%C3%ADsl,50,4,0,,0,0,2013-03-08T14:34:58Z,"Flokkskvísl, sem gengur einnig undir nöfnunum riðill, sveit og lið, er hernaðareining sem samans...","'''Flokkskvísl''', sem gengur einnig undir nöfnunum '''riðill''', '''sveit''' og '''lið''', er [...",flokkskvísl
11586,27685,Bandamenn (seinni heimsstyrjöldin),https://is.wikipedia.org/wiki/Bandamenn_%28seinni_heimsstyrj%C3%B6ldin%29,500,103,0,,1,0,2024-04-10T11:59:13Z,\nBandamenn í Síðari heimsstyrjöldinni voru þau ríki sem börðust gegn Öxulveldunum. Til bandaman...,"[[Mynd:Teheran conference-1943.jpg|thumb|right|Hinir „þrír stóru“: [[Jósef Stalín]], [[Franklin ...",bandamenn (seinni heimsstyrjöldin)
14391,33641,Frida Kahlo,https://is.wikipedia.org/wiki/Frida_Kahlo,411,31,0,,3,0,2024-07-09T01:08:56Z,\nFrida Kahlo (6. júlí 1907 – 13. júlí 1954) var mexíkósk listakona sem þróaði sérstakan stíl þa...,"[[Mynd:Frida_Kahlo,_by_Guillermo_Kahlo.jpg|thumb|right|Frida Kahlo á ljósmynd eftir föður hennar...",frida kahlo
28479,65519,Matarhefð,https://is.wikipedia.org/wiki/Matarhef%C3%B0,57,21,0,,1,0,2013-07-03T09:28:26Z,Matarhefð eða matarmenning er safn af hefðum og tækni í kringum matreiðslu og varðeislu matvæla ...,'''Matarhefð''' eða '''matarmenning''' er safn af [[hefð]]um og [[tækni]] í kringum [[matreiðsla...,matarhefð
54036,127265,Stríð Rússlands og Úkraínu,https://is.wikipedia.org/wiki/Str%C3%AD%C3%B0_R%C3%BAsslands_og_%C3%9Akra%C3%ADnu,291,38,0,,9,6,2024-08-17T10:41:54Z,"\nRússnesk-úkraínska stríðið er viðvarandi og langvinn átök sem hófust í febrúar 2014, þar sem e...",[[Mynd:2022 Russian invasion of Ukraine.svg|thumb|Hernaðarástand í febrúar 2022]]\n'''Rússnesk-ú...,stríð rússlands og úkraínu
55644,132275,Zootropolis,https://is.wikipedia.org/wiki/Zootropolis,214,61,0,,4,1,2023-09-24T10:49:47Z,\n\nZootropolis (enska: Zootopia) er bandarísk teiknimynd framleidd af Walt Disney Animation Stu...,{{DISPLAYTITLE:''Zootropolis''}}\n{{Kvikmynd\n| nafn = Zootropolis\n| upprunalegt h...,zootropolis


In [15]:
s1 = "(What's The Story) Morning Glory?"
s2 = "(What's the Story) Morning Glory?"